## submit jobs to cluster

In [2]:
import os
from rosemary import jpt_in_notebook, jpt_setup; jpt_setup()
from llm.submit import submit_job, multiline_to_singleline, shell_scripts_template_slurm

first_N = None
embed_type = 'text_embedding'
log_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/'

test_run = 1
test_run = bool(test_run)

####
md = 'mpnet'; model_name = 'all-mpnet-base-v2'; encode_fn_type = 'input'; normalize_embeddings = True

dataset = 'wizardlm'
## 
clustering_fn_list = []
# clustering_fn_list += [
#     f'cl=kmeans_nc={n_clusters}' for n_clusters in [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# ]

clustering_fn_list = [
    f"cl=kmeans_md={md}_dist=cd_emb={'+'.join(embed_type.split('_'))}_nc={n_clusters}"
]

# first_N = 10_000
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}'
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}_bsz=256'


###

cmds = []
for clustering_fn in clustering_fn_list:
    save_dir = (f"/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/"
                f"clustering/{encode_fn_type+'/' if encode_fn_type!='sft' else ''}{model_name}/"
                f"{dataset+'(N='+str(first_N)+')' if first_N else dataset}/{clustering_fn}")

    cmd = f"""
    python note_pruning_clustering.py \
        --model_name {model_name} \
        --dataset {dataset} \
        --encode_fn_type {encode_fn_type} \
        --clustering_fn {clustering_fn} \
        --embed_type {embed_type} \
        {'--normalize_embeddings' if normalize_embeddings else ''} \
        {'--first_N '+str(first_N) if first_N else ''} \
        --save_dir {save_dir} \
    """.strip()
    cmd = multiline_to_singleline(cmd)

    shell_scripts = shell_scripts_template_slurm.format(
        conda_env='open-instruct',
        cwd=os.getcwd(),
        cmd=cmd,
        log_dir=log_dir,
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=f'cluster.{dataset}.{clustering_fn}', 
        nodes=1,
        num_cpus=64,
        cpu_mem=128,
        num_gpus=1,
        gpu_type='v100',
        test_run=test_run,
        job_duration=6,
    )
    print(cmd)
    cmds.append(cmd)
    
print(f'#cmds: {len(cmds)}')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



Submiting job with:
{
    "job_name": "cluster.wizardlm.cl=kmeans_nc=100",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": true,
    "queue": "x86_6h",
    "num_jobs": 1
}
python note_pruning_clustering.py --model_name all-mpnet-base-v2 --dataset wizardlm --encode_fn_type input --clustering_fn cl=kmeans_nc=100 --normalize_embeddings --save_dir /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/clustering/input/all-mpnet-base-v2/wizardlm/cl=kmeans_nc=100

Submiting job with:
{
    "job_name": "cluster.wizardlm.cl=kmeans_nc=200",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": true,
    "queue": "x86_6h",
    "num_jobs": 1
}
python note_pruning_clustering.py --model_name all-mpnet-base-v2 --dataset wizardlm --encode_fn_type input --clustering_fn cl=kmeans_nc=200 --normalize_embeddings --save_dir /gpfs/u/home/PTFM/PTFMqngp/

In [8]:

with open('note_pruning_cluster_test.sh', 'w') as f:
    f.write('\n'.join(['CUDA_VISIBLE_DEVICES=1 '+x for x in cmds]))
    
    

## analyze clustering results

In [3]:
save_dir = (f"/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/"
            f"clustering/{encode_fn_type+'/' if encode_fn_type!='sft' else ''}{model_name}/"
            f"{dataset+'(N='+str(first_N)+')' if first_N else dataset}/{clustering_fn}")

In [200]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from note_pruning_clustering import clustering_cmp_clustering_fn

dataset='wizardlm'

dfs = []

model_name='all-mpnet-base-v2'; encode_fn_type = 'input'
clustering_fn_list = os.listdir(os.path.join('clustering', encode_fn_type, model_name, dataset))
clustering_fn_list = [x for x in clustering_fn_list if 'faiss' in x]
df = clustering_cmp_clustering_fn(clustering_fn_list, dataset, model_name, encode_fn_type)
dfs.append(df)

# model_name = 'llama-7b+lora:r=256:a=256'; encode_fn_type = 'sft'
# clustering_fn_list = os.listdir(os.path.join('clustering', encode_fn_type, model_name, dataset))


model_name = 'bge-large-en-v1.5'; encode_fn_type = 'input'
clustering_fn_list = os.listdir(os.path.join('clustering', encode_fn_type, model_name, dataset))
clustering_fn_list = [x for x in clustering_fn_list if 'faiss' in x]
df = clustering_cmp_clustering_fn(clustering_fn_list, dataset, model_name, encode_fn_type)
dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df

,n_clusters,dataset,model_name,clustering_fn,time_elapsed,inertia$\downarrow$,silhouette_score_cd$\uparrow$,silhouette_score_l2$\uparrow$,variance_ratio$\uparrow$,davies_bouldin_index$\downarrow$,cluster_sizes_min,cluster_sizes_mean,cluster_sizes_max,cent_dist_l2_vs_cd_spearmanr
0,200,dataset,all-mpnet-base-v2,cl=kmeansfaisscd_nc=200,25.072,127678.469,0.057,0.031,225.686,3.914,233,715,1539,NaN
1,200,dataset,all-mpnet-base-v2,cl=kmeansfaissl2_nc=200,26.751,98689.016,0.051,0.028,227.116,3.951,240,715,2170,NaN
2,200,dataset,bge-large-en-v1.5,cl=kmeansfaisscd_md=bge_dist=cd_emb=text+embed...,17.438,67329.070,0.042,0.023,227.575,4.303,35,715,1730,1.0
3,200,dataset,bge-large-en-v1.5,cl=kmeansfaissl2_md=bge_dist=l2_emb=text+embed...,16.831,59282.949,0.041,0.022,228.234,4.284,35,715,1687,1.0


## clustering code base


In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    print(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0] 
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
# !nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/minoconda3_x86/envs/open-instruct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['0', '5']
0


In [2]:
import re
import argparse
import json
import pickle
import string
import time
import matplotlib.pyplot as plt

import numpy as np
import scipy
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.cluster import KMeans, AgglomerativeClustering
import torch
from tqdm import tqdm
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

import time
from note_pruning_analysis import (
    get_lm_output,
    get_dataset,
)
from note_pruning_clustering import (
    pairwise_cosine_distance,
    clustering_sort_by_cluster_size,
    clustering_algorithm_scores,
    clustering_dist_to_centroids,
    clustering_knn_withincluster,
    clustering_run,
    clustering_compute_and_save_results,
    sklearn_compute_inertia_dense,
)

from llm.submit import get_host_info
if get_host_info()['arch'] == 'x86_64': import faiss

In [188]:
# md = 'mpnet'; model_name = 'all-mpnet-base-v2'; encode_fn_type = 'input'; normalize_embeddings = True
md = 'bge'; model_name = 'bge-large-en-v1.5'; encode_fn_type = 'input'; normalize_embeddings = True
# md = 'llama7b'; model_name = 'llama-7b+lora:r=256:a=256'; encode_fn_type = 'sft'; normalize_embeddings = True

dataset = 'wizardlm'

# first_N = 10_000
first_N = None

# clustering_fn = f'cl=kmeans_nc={n_clusters}'
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}_bsz=4096'
# clustering_fn = f'cl=kmeansfaissl2_nc={n_clusters}'
# clustering_fn = f'cl=kmeansfaisscd_nc={n_clusters}'

n_clusters = 200

embed_type = 'text_embedding'
# embed_type = 'grad_rp_loraB'
# clustering_fn = f"cl=kmeansfaisscd_md={md}_dist=cd_emb={'+'.join(embed_type.split('_'))}_nc={n_clusters}"
clustering_fn = f"cl=kmeansfaissl2_md={md}_dist=l2_emb={'+'.join(embed_type.split('_'))}_nc={n_clusters}"


In [189]:
save_dir = os.path.join('clustering', encode_fn_type, model_name, dataset, clustering_fn+'_test')
os.makedirs(save_dir, exist_ok=True)
save_dir

'clustering/input/bge-large-en-v1.5/wizardlm/cl=kmeansfaissl2_md=bge_dist=l2_emb=text+embedding_nc=200_test'

In [190]:

ds = get_dataset(dataset, processed=True)
if encode_fn_type == 'input':
    def get_user_prompt_fn(example):
        example['text'] = example['messages'][0]['content']
        return example
    ds = ds.map(get_user_prompt_fn, num_proc=16)
elif encode_fn_type == 'sft':
    def combine_first_turn_conv_fn(example):
        example['text'] = "[USER]"+" "*20+example['messages'][0]['content']+" "*20+\
                          "[ASSISTANT]"+" "*20+example['messages'][1]['content']
        return example
    ds = ds.map(combine_first_turn_conv_fn, num_proc=16)

ds

Dataset({
    features: ['dataset', 'id', 'messages', 'text'],
    num_rows: 143000
})

In [191]:
d = get_lm_output(dataset, 
                  model_name, 
                  encode_fn_type=encode_fn_type,
                  return_text_embedding=True,)

if embed_type not in ['text_embedding', 'grad_rp_loraB']:
    raise ValueError(f'embed_type={embed_type} not supported.')
X = d[embed_type]
if normalize_embeddings:
    X = X / np.maximum(np.linalg.norm(X, axis=-1, keepdims=True), 1e-8) # possibly divide by zero.
nan_rows = np.isnan(X).any(-1)
if nan_rows.any():
    print(f'Found {nan_rows.sum()} ({100*nan_rows.sum()/len(X)} pct) nan rows in embeddings. ')
    
if first_N:
    ds = ds.select(range(first_N))
    X = X[:first_N]

In [192]:


info = {}
info['N'] = len(X)
info['dataset'] = 'dataset'
info['model_name'] = model_name
info['encode_fn_type'] = encode_fn_type
info['clustering_fn'] = clustering_fn

t0 = time.time()
Y, C = clustering_run(clustering_fn, X)
info['time_elapsed'] = time.time()-t0
info['scores'] = {}
info['scores'].update({'inertia': sklearn_compute_inertia_dense(X, Y, C)})
info['scores'].update(clustering_algorithm_scores(X, Y))

df = clustering_compute_and_save_results(X, Y, C, ds=ds, save_dir=save_dir)
info['cent_dist_l2_vs_cd_spearmanr'] = float(scipy.stats.spearmanr(
    df['cent_dist_l2'].to_numpy(), df['cent_dist_cd'].to_numpy()).statistic)
info['cluster_sizes'] = np.unique(Y, return_counts=True)[1].tolist()

with open(os.path.join(save_dir, 'info.json'), 'w') as f:
    json.dump(info, f, ensure_ascii=False, indent=4)


Sampling a subset of 51200 / 143000 for training
Clustering 51200 points in 1024D to 200 clusters, redo 1 times, 30 iterations
  Preprocessing in 0.23 s
  Iteration 29 (15.26 s, search 13.49 s): objective=21083.8 imbalance=1.102 nsplit=0       
